# Housing Price Index Change in the City of Los Angeles from 2010 to 2019

## **Research Question**: Do parks in the city of Los Angeles have an effect on housing prices and change in tenure from 2010 to 2019?

My portion of the assignment is looking at housing prices in the LA. I used the 2010 and 2019 FHFA Annual House Price Indexes to calculate the percent change in housing prices in the City of Los Angeles. This data is in census tracts. We were curious to see if there is a relationship between access to parks and housing price trends. If there is a concentration of housing prices increasing in an area with parks, then nearby parks may be a factor that increases home values, which may be inducing gentrification.

FHFA Housing Price Index:
- https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx

ACS Data on Tenure:
- [ACS 5-YR Table B25003](https://censusreporter.org/data/table/?table=B25003&geo_ids=16000US0644000,140|16000US0644000&primary_geo_id=16000US0644000)
- [Download Link](https://api.dokku.censusreporter.org/1.0/data/download/acs2019_5yr?table_ids=B25003&geo_ids=16000US0644000,140|16000US0644000&format=geojson)

Trust for Public Land's ParkScore Map
- https://www.tpl.org/city/los-angeles-california
- https://www.tpl.org/parkserve/downloads

### I've broken up my notebook into two parts because the kernel kept crashing when it was one notebook. 
## Importing and Cleaning Up Housing Price Data & Park Data
[Sim's midterm notebook for reference](https://github.com/simbun/up206a-sim/blob/main/midterm/midterm.ipynb)

In [1]:
import pandas as pd

# to read and visualize spatial data
import geopandas as gpd

# to provide basemaps 
import contextily as ctx

# to give more power to your figures (plots)
import matplotlib.pyplot as plt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


`geofile` is a geojson from the ACS that has census tract data. the `fhfa` is a csv with the housing price index data from the FHFA. I merge these two files together so the data can be mapable. 

In [2]:
# the geofile is file with the geometry element 
geofile = gpd.read_file('acs2019_5yr_B25003_14000US06037222001.geojson')

# the fhfa is the csv file with the data i want to map
fhfa = gpd.read_file('FHFA_HPI_Fixed.csv')

Although I am just using the geofile for the geometry aspect, I still would like to clean up the data.

In [3]:
geofile. info(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   geoid             1005 non-null   object  
 1   name              1005 non-null   object  
 2   B25003001         1005 non-null   float64 
 3   B25003001, Error  1005 non-null   float64 
 4   B25003002         1005 non-null   float64 
 5   B25003002, Error  1005 non-null   float64 
 6   B25003003         1005 non-null   float64 
 7   B25003003, Error  1005 non-null   float64 
 8   geometry          1005 non-null   geometry
dtypes: float64(6), geometry(1), object(2)
memory usage: 70.8+ KB


In [4]:
# listing what columns to keep 
columns_to_keep= ['geoid','name','B25003001','B25003002','B25003003', 'geometry']
# applying the function
geofile=geofile[columns_to_keep]

I am pulling the last two rows of data because I already know from experience that the last row is not census tract data.

In [5]:
geofile.tail(2)

,geoid,name,B25003001,B25003002,B25003003,geometry
1003,14000US06037990200,"Census Tract 9902, Los Angeles, CA",0.0,0.0,0.0,"MULTIPOLYGON (((-118.63598 34.03255, -118.6325..."
1004,16000US0644000,"Los Angeles, CA",1383869.0,509504.0,874365.0,"MULTIPOLYGON (((-118.66818 34.18987, -118.6681..."


In [6]:
# dropping this row because it is not a census tract
geofile=geofile.drop([1004])
# making sure it is dropped
geofile.tail(1)

,geoid,name,B25003001,B25003002,B25003003,geometry
1003,14000US06037990200,"Census Tract 9902, Los Angeles, CA",0.0,0.0,0.0,"MULTIPOLYGON (((-118.63598 34.03255, -118.6325..."


I am now cleaning up the fhfa data and I am spending more time on this because this is a dataset that I am focusing on. 

In [7]:
fhfa.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1640 entries, 0 to 1639
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   addition to geoid  1640 non-null   object  
 1   geoid              1640 non-null   object  
 2   field_3            1640 non-null   object  
 3   hpi 2010           1640 non-null   object  
 4   hpi 2019           1640 non-null   object  
 5   % change           1640 non-null   object  
 6   geometry           0 non-null      geometry
dtypes: geometry(1), object(6)
memory usage: 89.8+ KB


In [8]:
# listing what columns to keep 
columns_to_keep= ['field_3','hpi 2010','hpi 2019','% change','geometry']

In [9]:
# applying the function
fhfa=fhfa[columns_to_keep]

In [10]:
# getting the list for the fhfa columns so i can copy and paste it later and rename it
list(fhfa)

['field_3', 'hpi 2010', 'hpi 2019', '% change', 'geometry']

In [11]:
# renaming the columns
fhfa.columns= ['geoid', 'hpi 2010', 'hpi 2019', 'pc_hpi', 'geometry']

In [12]:
# We cannot map objects so we must change them into floats, except for the geoid column
fhfa['hpi 2010'] = fhfa ['hpi 2010'].astype(float)
fhfa['hpi 2019'] = fhfa ['hpi 2019'].astype(float)
fhfa['pc_hpi'] = fhfa ['pc_hpi'].astype(float)

After cleaning up the datasets, I am merging the datat using the column, 'geoid'. This will make the fhfa dataset mappable. 

In [13]:
# merging the data
fhfa=fhfa.merge(geofile,on= 'geoid')

In [14]:
# checking to see if it worked
fhfa.tail(1)

,geoid,hpi 2010,hpi 2019,pc_hpi,geometry_x,name,B25003001,B25003002,B25003003,geometry_y
574,14000US06037297602,304.19,435.04,0.43,None,"Census Tract 2976.02, Los Angeles, CA",1712.0,700.0,1012.0,"MULTIPOLYGON (((-118.29286 33.72110, -118.2928..."


It worked. Now I am cleaning up the data further, removing the columns from the geoid. I should have done this sooner but here we are.

In [15]:
# naming the columns i need to keep
columns_to_keep= ['geoid','hpi 2010','hpi 2019','pc_hpi','name','geometry_y']

In [16]:
# executing the function 
fhfa=fhfa[columns_to_keep]

In [17]:
# listing the column names so i can copy and paste
# i only need to change the geometry_y to geometry
# Ariana said if i didn't drop the y it would give me a hard time later 
list(fhfa)

['geoid', 'hpi 2010', 'hpi 2019', 'pc_hpi', 'name', 'geometry_y']

In [18]:
# renaming the columns
fhfa.columns = ['geoid', 'hpi 2010', 'hpi 2019', 'pc_hpi', 'name', 'geometry']
# checking to see if it worked 
fhfa.head(1)

,geoid,hpi 2010,hpi 2019,pc_hpi,name,geometry
0,14000US06037101110,505.95,934.82,0.85,"Census Tract 1011.10, Los Angeles, CA","MULTIPOLYGON (((-118.30229 34.25870, -118.3009..."


The other main dataset I am working with is the parks data. 

In [ ]:
# importing the parks data
parks = gpd.read_file('SoCalParks.zip')
parks.to_file('SoCalParks.geojson', driver='GeoJSON')

In [ ]:
# changing the followng data columns to strings
parks['Park_Name'] = parks ['Park_Name'].astype(str)
parks['Park_Urban'] = parks ['Park_Urban'].astype(str)
parks['Park_Desig'] = parks ['Park_Desig'].astype(str)
parks['Park_Owner'] = parks ['Park_Owner'].astype(str)
parks['Park_Local'] = parks ['Park_Local'].astype(str)
parks['Park_Manag'] = parks ['Park_Manag'].astype(str)
parks['Park_Loc_1'] = parks ['Park_Loc_1'].astype(str)
parks['Park_Statu'] = parks ['Park_Statu'].astype(str)
parks['Park_Est_D'] = parks ['Park_Est_D'].astype(str)
parks['Park_Addre'] = parks ['Park_Addre'].astype(str)
parks['Park_State'] = parks ['Park_State'].astype(str)
parks['Park_Sta_1'] = parks ['Park_Sta_1'].astype(str)
parks['Park_Count'] = parks ['Park_Count'].astype(str)   
parks['Park_Cou_1'] = parks ['Park_Cou_1'].astype(str)     
parks['Park_Place'] = parks ['Park_Place'].astype(str)
parks['Park_Pla_1'] = parks ['Park_Pla_1'].astype(str)   
parks['Park_Urb_1'] = parks ['Park_Urb_1'].astype(str)     
parks['Park_Zip'] = parks ['Park_Zip'].astype(str)
parks['Park_Bound'] = parks ['Park_Bound'].astype(str)   
parks['Park_Sourc'] = parks ['Park_Sourc'].astype(str)     
parks['Park_Feedb'] = parks ['Park_Feedb'].astype(str)
parks['Park_DateA'] = parks ['Park_DateA'].astype(str) 
parks['DataShare_'] = parks ['DataShare_'].astype(str)

Next, I am mapping the percent change of the housing price and the parks in Los Angeles City. I am layering these two maps on top of each other to see where the parks and housing prices changes are. Ariana and I suspect that parks may have an influence on the housing prices. If we are right, we would see that the areas with the highest increase in housing prices are in close to proximity to parks.

In [ ]:
# addressing the error as there is no Dataframe
from geopandas import GeoDataFrame

# updating the fhfa data so it is a dataframe
fhfa = GeoDataFrame(fhfa)

parks_web_mercator = parks.to_crs(epsg=3857)
fhfa_web_mercator = fhfa.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(15, 15))

# add the layer with ax=ax in the argument 
fhfa_web_mercator.plot(ax=ax,column='pc_hpi', cmap='OrRd', legend=True,alpha=0.8)
parks_web_mercator[parks_web_mercator['Park_Place'] == 'Los Angeles city'].plot(ax=ax, color="darkgreen",alpha= 0.8)
                                            
# turn off axis
ax.axis ('off')
                                            
#set a title
ax.set_title('Parks in LA City Compared to Percent Change in Housing Price Index', fontsize=24,pad=20)

#add basemap
ctx.add_basemap(ax)

The areas that are more intensely red have the highest increases in housing prices, in terms of percentage. Note, for the legend the highest is 2.00 but please read it as 200%. So this map shows that the area with the highest increas in housing prices is in South LA and Central LA. However, these areas are lacking in parks. At a first glance, it seems we may have missed the mark with our assumption. However, even if the highest increase in housing prices is not close to parks, let's see if there is some statstical significance for our other data points.

For the next part, we are starting the spatial autocorrelation work. 

# Spatial Autocorrelation

This will be my attempt to do spatial autocorrelation for housing price index in Los Angeles Census Tracts.

In [ ]:
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

How many parks are there in each census block group? In order to answer this question, one must count the number of arrests that fall within each block group boundary. To do so, we conduct a spatial join.

Do parks have an impact on percent change in housing prices? Does the number of parks in a census tract affect whether or not there is a negative/positive change in housing prices in an area? Is that value statistically significant?

In [ ]:
# updating the dataset so it is a dataframe
fhfa = GeoDataFrame(fhfa) # i did this one already but i think it should be good to have it all in one place
parks= GeoDataFrame(parks)

fhfa = fhfa.to_crs(epsg=3857)
parks = parks.to_crs(epsg=3857)

In [ ]:
# now I am joining the parks and fhfa dataset. 
# from my understanding this is different from merging as we are not merging it using a column.
parks_join = gpd.sjoin(parks,fhfa, how='left')

# checking to see if the function worked
parks_join.head(2)

So, I got stuck at this point when I followed the lab. When I had it as `join = gpd.sjoin(parks,fhfa, how='left')`, later on when I tried to join the `parks_by_ct` to the parks data, it would say error and that the data does not exist. When I looked at Ariana's notebook, I noticed she did not have the same problem. I was stumpt!! It took a lot of troubleshooting to figure out this simple solution but here we are.

Next, we create another dataframe that counts parks by their corresponding census tracts:

In [ ]:
parks_by_ct = parks_join.geoid.value_counts().rename_axis('geoid').reset_index(name='parks_count')

In [ ]:
parks_by_ct.head(3)

The census tracts with the highest park count is 9.

In [ ]:
parks_by_ct[:20].plot.bar(figsize=(20,4),
                          x= 'geoid',
                          y='parks_count')

In [ ]:
# if you read my markdown cell earlier, this is where i got stuck. x(
# now we are merging the data using the geoid column.
parks=parks_join.merge(parks_by_ct, on ='geoid') 

In [ ]:
# seeing if the merge worked. 
parks.sort_values(by="parks_count").tail(5)

We see that the census tracts with the highest park count is in Santa Monica Mountains Conservancy which makes sense. Though I am not really sure if there are homes there ( I have not been to Santa Monica Mountains) but the housing price changed by about 47% from 2010 to 2019. 

## Normalizing: Parks per 10 percentage change of housing price index
Rather than proceeding with an absolute count of parks, we are normalizing it by number of percent change of housing prices in the census tract. I thought it would be good to use 10% as an increment of measure since 100% may be too much. 

So I am attempting to normalize the number of parks by every 10% of change of housing price. That is what I am attempting but if I am not doing it correctly, please let me know because I am a bit confused. 


In [ ]:
parks['parks_per_10'] = parks['parks_count']/(parks['pc_hpi'])*.10

In [ ]:
parks.sort_values(by="parks_per_10").head(2)

Right now `pc_hpi` is a decimal but I am reading it as a percentage. So, if it reads 1.68, it is a 168% change. Should I be chang the `pc_hpi` column?


Here, we sort the values by descending arrest rate, and only show a slice of the data, the top 20 geographies using the handy `[:20]`.

In [ ]:
fig,ax = plt.subplots(figsize=(12,10))
parks.sort_values(by='parks_per_10',ascending=False)[:20].plot(ax=ax,
                                                                 color='darkgreen',
                                                                 edgecolor='white',
                                                                 alpha=0.5,legend=True)


# title
ax.set_title('Top 20 locations of Green Spaces per 10 Percent Change in Housing Price')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
# get the bounding box coordinates for the arrest data
minx, miny, maxx, maxy = parks.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

Adding the boundaries so we can see the full map

In [ ]:
fig,ax = plt.subplots(figsize=(12,10))
parks.sort_values(by='parks_per_10',ascending=False)[:20].plot(ax=ax,
                                                                 color='darkgreen',
                                                                 edgecolor='white',
                                                                 alpha=0.9,legend=True)


# title
ax.set_title('Top 20 Locations of Green Spaces per 10 Housing Price Percent Changes')

# no axis
ax.axis('off')

# use the bounding box coordinates to set the x and y limits
ax.set_xlim(minx - 10, maxx + 10) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 10, maxy + 10)

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

I wonder if the data is skewed looking at this map since there is such a concentration of parks in the Santa Monica Mountains.

## Choropleth map of parks

We ready to generate a choropleth map of parks. 

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))

parks.plot(ax=ax,
        column='parks_per_10',
        legend=True,
        alpha=0.8,
        cmap='RdYlGn_r',
        scheme='quantiles')

ax.axis('off')
ax.set_title('Parks per 10 Percent Change',fontsize=22)
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

I believe this map is showing us that there is concentration of parks in the West, Northwest, and Northeast. For the next map, I will be incorporating the housing price percent change layer and comparing the concentration of parks.

In [ ]:
parks_web_mercator = parks.to_crs(epsg=3857)
fhfa_web_mercator = fhfa.to_crs(epsg=3857)


# add the layer with ax=ax in the argument 
fig,ax = plt.subplots(figsize=(12,10))

fhfa_web_mercator.plot(ax=ax,column='pc_hpi', cmap='OrRd', legend=True,alpha=0.8)


parks.plot(ax=ax,
        column='parks_per_10',
        legend=True,
        alpha=0.8,
        cmap='PiYG',
        scheme='quantiles')

# title
ax.set_title('Parks per 10 Percent Change')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Again, we see that the areas with the highest change in housing prices are not in areas with high concentrations of parks. In other words, the resulting map tells us that there does not appear to be spatial clusters of parks where high housing price changes are more prevalent. However, we cannot statistically back this up. 

## Please find my work on spatial weight, spatial lag, and etc in part 2.